In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

class custom_callback(tf.keras.callbacks.Callback):
    def __init__(self,x_val,y_val):
        super().__init__()
        self.x_val = x_val
        self.y_val = y_val
        self.f1_s=[]
        self.AUC=[]
        self.val_acc=[]
        self.val_loss=[]
        pass
        
    def on_epoch_end(self, epoch, logs={}):
        super().on_train_begin()
        #Getting the validation prediction
        y_pred=tf.reshape((self.model.predict(self.x_val)),(1,-1)).numpy()[0]
        y_pred[y_pred==-1]=0
        y_pred=y_pred.astype('int32')
        y_true=self.y_val.astype('int32')
        
        #getting the f1 score
        f1=tfa.metrics.F1Score(num_classes=2, threshold=0.5,average='micro')       
        metric.update_state(y_true, y_pred)
        f1_score = metric.result().numpy()
        print('F1 Score is ',f1_score)
        self.f1_s.append(f1_score)
        
        #getting the AUC Score
        a=tf.keras.metrics.AUC(num_thresholds=len(self.y_val))
        a.update_state(y_true,y_pred)
        auc=a.result().numpy()
        print('AUC Score is ',auc)
        self.AUC.append(auc)
        
        #Getting the accuracy score & log_loss
        validation_acc=accuracy_score(y_true, y_pred)
        validation_loss=log_loss(y_true, y_pred)
        #print(validation_acc,validation_loss)
        
        #Appending these values to the main list
        self.val_acc.append(validation_acc)
        self.val_loss.append(validation_loss)
        
        if epoch>=2:
            if self.val_acc[-1]>self.val_acc[-2]:
                file_path='model_save'
                monitor='val_acc'
                tf.keras.callbacks.ModelCheckpoint(file_path,monitor)
                
        #Updating the learning rate
        lr=float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        
        if epoch>=2:
            if self.val_acc[-1]<self.val_acc[-2]:
                lr=lr*0.9
            if epoch%3==0:
                lr=lr*0.95
        tf.keras.backend.set_value(self.model.optimizer.learning_rate,lr)
        #Found this syntax in https://stackoverflow.com/questions/69758568/how-to-change-learning-rate-in-tensorflow-after-a-batch-end
        #print(self.model.optimizer.learning_rate)
        
        #Checking if the val_Acc has increased and then stopping training if not improving
        if epoch>=3:
            if all([self.val_acc[-1]>self.val_acc[-2],self.val_acc[-1]>self.val_acc[-3]]):
                self.model.stop_training=True
        
        #https://stackoverflow.com/questions/64806541/performing-np-isnan-on-keras-model-weighta-fails-with-typeerror-ufunc-isnan-n
        #Code to check if any of the weights is Nan
        if (logs.get("loss")==None) or np.any([np.any(np.isnan(x)) for x in self.model.get_weights()]):
            self.model.stop_training=True
        pass
    
#https://towardsdatascience.com/custom-callback-functions-for-transformers-ae65e30c094f 
#https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras